In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("openai/gpt-oss-20b")
# model = AutoModelForCausalLM.from_pretrained(
#     "openai/gpt-oss-20b",
#     torch_dtype=torch.bfloat16,
# ).to('cuda:1')



/home/tikhonov/.conda/envs/condaenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
messages = [
    {"role": "system", "content": "YOYOYOYOYUOY"},
    {"role": "user", "content": "341*6432"},
]
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
)

print(repr(tokenizer.decode(inputs["input_ids"][0], skip_special_tokens=False)))

'<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.\nKnowledge cutoff: 2024-06\nCurrent date: 2025-09-17\n\nReasoning: medium\n\n# Valid channels: analysis, commentary, final. Channel must be included for every message.<|end|><|start|>developer<|message|># Instructions\n\nYOYOYOYOYUOY\n\n<|end|><|start|>user<|message|>341*6432<|end|><|start|>assistant'


In [ ]:
# 1. Формируем историю с помощью apply_chat_template
messages = [

    {"role": "user", "content": "341*6432"}
]

# Получаем строку со всеми системными промптами и историей, но без промпта для генерации
prompt_history = tokenizer.apply_chat_template(messages, add_generation_prompt=False, tokenize=False)

# 2. Вручную добавляем "подставные" мысли, НЕ закрывая теги
# Возьмем часть рассуждений из первого рабочего примера
qwen_reasoning_part = "We are a language model. The user gives a multiplication problem \"341*6432\". They likely want the product. So compute 341 *"
manual_part = f"<|start|>assistant<|channel|>analysis<|message|>{qwen_reasoning_part}"

full_prompt_string = prompt_history + manual_part

print("--- Manually constructed prompt string ---")
print(repr(full_prompt_string))
print("\n--- Tokenizing and generating ---")

# 3. Токенизируем и генерируем
inputs = tokenizer(full_prompt_string, return_tensors="pt").to(model.device)

outputs = model.generate(**inputs, max_new_tokens=2048)

# 4. Декодируем только новую, сгенерированную часть
generated_part = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=False)

print("\n--- Model Generation (what it added) ---")
print(generated_part)



--- Manually constructed prompt string ---
'<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.\nKnowledge cutoff: 2024-06\nCurrent date: 2025-09-17\n\nReasoning: medium\n\n# Valid channels: analysis, commentary, final. Channel must be included for every message.<|end|><|start|>user<|message|>341*6432<|end|><|start|>assistant<|channel|>analysis<|message|>We are a language model. The user gives a multiplication problem "341*6432". They likely want the product. So compute 341 *'

--- Tokenizing and generating ---

--- Model Generation (what it added) ---
 6432. Let's do it manually: 6432 * 341 = 6432 * 300 + 6432 * 40 + 6432 * 1. 6432*300 = 1,929,600 (since 6432*100=643200, times 3). 6432*40 = 257,280. 6432 = 6,432. Sum: 1,929,600 + 257,280 = 2,186,880. Then add 6,432: Total 2,193,312. Alternatively more precisely: Let's multiply: 6432 * 341. Check: 6432 * 341: 6432 * (300 + 40 + 1) as above. 6432*300=1,929,600. 6432*40=257,280. 6432*1=6,432. Add: 1,929,6